(cont:mm:dynamics)=

This chapter describes the overall memory management paging dynamics once the system is up and running.
How memory is freed at boot time, how its allocated and used by the kernel and user space/buffer cache
and how its constantly reclaimed on an actively running system.


# Paging Dynamics
```{figure} ../images/mm/PGdymanics.png
---
width: 100%
name: fig:PGdymanics
---
Paging dynamics of a Linux system
```
## Boot-time
When a paged virtual memory system is booted one for the first actions is to insert every page of physical memory onto a free page list.  As the boot sequence continues the kernel needs to allocate memory for kernel text and data therefore some of the free pages must be allocate from the free list.  By the time the boot sequence is complete typically 10%-15% of the total memory will used by the kernel for the life the running system.
## Allocating and consuming physical memory
### Kernel page wiring
Most or all of the memory pages allocated by the kernel especially during the boot sequence are permanent in other words will never be released or freed to be used any other process, these are known as wired pages.  However, as the system runs and new processes, files and network connections are created, etc. the kernel temporarily allocates memory for data structures.  When processes exit, files are deleted and network connections are terminated the associated data structures are freed and the kernel frees the underlying memory pages.  Over time its common for the kernel to consume and wire between 10% and 25% of the total memory depending on the number and types of applications running on the system.
  
### User Page faults
The virtual address space of a process is typically very large and but sparsely utilized.  For example, the typical x86_64 address space is fixed in size, hundreds of TB in size but typically only a few GB of virtual memory regions are actually allocated and within those regions only a few MB of that are actually used and map physical memory.  Virtual memory is allocated in the address space by a process using either the sbrk() or mmap() system calls.  Newly allocated virtual memory has no physical memory mapped into it until the process references it and a page fault occurs.  The page fault handling routines allocate free pages of memory and map them into the virtual memory on demand.  This means that not only is the whole address space sparsely populated with valid virtual memory regions but even those regions are sparsely populated with physical memory.  Over time the collective page faults for every running process consume a significant amount of physical memory on the system.

### Buffer Cache
As was mentioned earlier in this chapter, the Buffer Cache uses free pages of physical memory to cache file system data and meta-data.  When a file is opened and any block of that file is read or written the file system will typically read several blocks of that file into free pages managed by the Buffer Cache, thereby eliminating most of the future synchronous IO associated with the file.  The Buffer Cache continues to cache a file's data and meta-data after the file is closed and is no longer being referenced in case the file is re-opened its data will cached in memory.

## Freeing physical memory
### Process Exiting
When a process exits all of the data structures associated with that process are freed.  This potentially frees several pages that the kernel wired for this process.  In addition all of the virtual memory regions within the processes virtual address space are free.  Since the anonymous regions are private and specific to the process all the physical memory pages mapped into these virtual regions are immediately freed.  However, the mapped file regions only contain mapped Buffer Cache pages so they are not freed until the Buffer Cache logic decides to do so.

### Page reclaiming
One of the major benefits of paged virtual memory is the capability of over committing physical memory.  Every process has a huge fixed size virtual address space that is sparsely populated with valid regions and they can be sparsely populated with physical memory mappings.  In addition, the buffer cache aggressively uses free memory to cache file system data.  After a system has been up for a while processes are created and many have exited.  These processes allocate memory, invoke page faults, open, read, write and close files, etc.  After a relatively short period of time free memory becomes exhausted and the memory management system must reclaim physical memory in order to satisfy additional requests.  At this point the system reaches a steady state of continually allocating and reclaiming physical memory.  Most systems implement a background daemon or process/thread that runs continually monitoring memory requests, locating the most ideal pages of memory to be reclaimed and doing the actual reclaiming of an adequate number of pages of physical memory.  Reclaiming memory involves using one or more of the reclamation algorithms discussed earlier and is generally grouped into 3 categories; buffer cache purging, paging out and swapping out.  Unlike page cache purging both paging out and swapping out involves unmapping and removing the physical memory that was mapped into anonymous virtual regions by page faults.  These pages must be written to a swap file of they were modified or dirty in order to maintain correctness.

#### Buffer cache purging
Buffer cache purging simply involves freeing the physical pages of memory that were used to cache file system data.  As was described in the Buffer Cache section, that code very aggressively uses all the available free memory to cache file system data.  Theoretically if a file is opened and used it will be used again in the near future, even if that file is closed it will be reopened and used.  In addition if one block of a file is used the surrounding blocks will also be used in the near future.  For these reasons the Buffer Cache is by far the largest consumer of physical memory and will therefore be the first place the memory reclaiming code will attempt to reclaim pages of physical memory.

Since the buffer cache is a write-back cache, writes to various locations in a file that is being cached might not transfer the data to the actual storage immediately.  Instead the modified file system data might be held in pages of free memory managed by the buffer cache until it gets flushed to storage.  The buffer cache pages that need to be flushed out before they can be reused are known as dirty buffer cache pages.  Buffer cache pages that were never written to or have already been flushed to storage are know as clean buffer cache pages.  For file system correctness clean buffer cache pages can immediately be reused but all dirty buffer cache pages must to be flushed to storage before they can be reused.  The buffer cache logic attempts to keep the majority of its pages clean by periodically flushing them to storage, therefore most can be reused immediately.

#### Paging out
Paging out is simply reclaiming a few physical pages of memory that are mapped into the virtual address spaces of one or more processes.  When the demand for free physical memory is fairly light paging out can be all that is needed to satisfy most or even all of this demand.  Since paging out only increases the supply and does not decrease the demand for free pages its likely the associated processes will continue to run and those freed pages will be reused and more paging out will be necessary.  

#### Swapping out
Swapping is the act of reclaiming most or all of the physical memory from large regions of one or more virtual address spaces.  When the demand for memory is high and paging does not satisfy enough pages to keep up with the demand the page reclaim code starts swapping.  In addition to reclaiming large amounts of physical memory swapping may also involve temporarily suspending processes that are being swapped out so they don't run and demand more memory until the system recovers from the memory demand.  

- System shutdown